In [1]:
import json
import pandas as pd

In [2]:
f = open('/home/dy/filerepo/Research/multiagent-human-experiment/mturk/experiment_data/mturk-experiment-4-20210826.json')
data = json.load(f)
f.close()

In [3]:
agents = [game[1] for game in data[0]['results']['data'][-1]['games']]
print(agents)

['sac_self_play_coordination_ring_0', 'nct_coordination_ring', 'action_cml_v_sac_none_no_ib', 'sac_v_bc_states_coordination_ring_0']


In [4]:
feedback_subtasks = [5, 7, 9, 11]
game_subtasks = [4, 6, 8, 10]
workers = range(len(data))

In [5]:
def get_subtask(agent, worker, game_subtasks, data):

    for game_subtask in game_subtasks: #find the right subtask for the agent order
        # if (data[worker]['results']['data'][game_subtask]['data']['trajectory'][0]['player_1_id'] == agent):
        if (data[worker]['results']['data'][game_subtask]['params']['playerOne'] == agent):
            return game_subtask
    return -1

def get_reward(worker, game_subtask, data):
    game_id = data[worker]['results']['data'][game_subtask]['trial_id']

    f = open('/home/dy/filerepo/Research/multiagent-human-experiment/server/data/server/' + game_id + '.json')
    game_data = json.load(f)    
    f.close()

    total_reward = 0

    # for timestep in data[worker]['results']['data'][game_subtask]['data']['trajectory']:
    for timestep in game_data['trajectory']:
        total_reward += timestep['reward']

    return total_reward

def get_actions(worker, game_subtask, data): #return actions per second
    game_id = data[worker]['results']['data'][game_subtask]['trial_id']

    f = open('/home/dy/filerepo/Research/multiagent-human-experiment/server/data/server/' + game_id + '.json')
    game_data = json.load(f)    
    f.close()

    count = 0
    STAY_ACTION = [0, 0]
    for i in range(len(game_data['trajectory'])):
        if(game_data['trajectory'][i]['joint_action'][0] != STAY_ACTION):
            count += 1

    return count / 60


In [6]:
score_data = pd.DataFrame()

all_scores = []
for worker in workers:
    scores_dict = {}
    for agent in agents:
        game_subtask = get_subtask(agent, worker, game_subtasks, data)
        reward = get_actions(worker, game_subtask, data)
        scores_dict[agent] = reward
        print(worker, agent, reward)
    all_scores.append(scores_dict)

0 sac_self_play_coordination_ring_0 1.1833333333333333
0 nct_coordination_ring 0.9666666666666667
0 action_cml_v_sac_none_no_ib 0.8666666666666667
0 sac_v_bc_states_coordination_ring_0 0.48333333333333334
1 sac_self_play_coordination_ring_0 4.983333333333333
1 nct_coordination_ring 5.166666666666667
1 action_cml_v_sac_none_no_ib 5.016666666666667
1 sac_v_bc_states_coordination_ring_0 5.716666666666667
2 sac_self_play_coordination_ring_0 3.6166666666666667
2 nct_coordination_ring 4.666666666666667
2 action_cml_v_sac_none_no_ib 4.033333333333333
2 sac_v_bc_states_coordination_ring_0 4.083333333333333
3 sac_self_play_coordination_ring_0 1.6
3 nct_coordination_ring 1.7833333333333334
3 action_cml_v_sac_none_no_ib 1.4333333333333333
3 sac_v_bc_states_coordination_ring_0 1.6833333333333333
4 sac_self_play_coordination_ring_0 6.15
4 nct_coordination_ring 9.033333333333333
4 action_cml_v_sac_none_no_ib 6.95
4 sac_v_bc_states_coordination_ring_0 8.883333333333333
5 sac_self_play_coordination_ri

In [43]:
actions_df = pd.DataFrame(all_scores)
actions_df['Total Average'] = actions_df.mean(axis=1)
actions_df

,sac_self_play_coordination_ring_0,nct_coordination_ring,action_cml_v_sac_none_no_ib,sac_v_bc_states_coordination_ring_0,Total Average
0,1.183333,0.966667,0.866667,0.483333,0.875000
1,4.983333,5.166667,5.016667,5.716667,5.220833
2,3.616667,4.666667,4.033333,4.083333,4.100000
3,1.600000,1.783333,1.433333,1.683333,1.625000
4,6.150000,9.033333,6.950000,8.883333,7.754167
5,1.000000,1.300000,1.283333,1.516667,1.275000
6,0.366667,0.516667,1.350000,0.883333,0.779167
7,4.333333,2.566667,0.933333,3.183333,2.754167
8,2.433333,3.933333,3.733333,4.000000,3.525000
9,3.083333,2.450000,3.316667,3.250000,3.025000


In [11]:
worker = 1
agent = 'nct_coordination_ring'
game_subtask = get_subtask(agent, worker, game_subtasks, data)

game_id = data[worker]['results']['data'][game_subtask]['trial_id']

f = open('/home/dy/filerepo/Research/multiagent-human-experiment/server/data/server/' + game_id + '.json')
game_data = json.load(f)    
f.close()

count = 0
STAY_ACTION = [0, 0]
for i in range(len(game_data['trajectory'])):
    if(game_data['trajectory'][i]['joint_action'][0] != STAY_ACTION):
        count += 1
    print(game_data['trajectory'][i]['joint_action'][0])

[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[-1, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[-1, 0]
[0, 1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
interact
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, -1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, -1]
[0, 0]
[0, 0]
[0, 0]
[0, -1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 0]
[0, 0]
[0, 0]
[0, -1]
[0, 0]
[0, 0]
[0, 0]
[0, -1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, -1]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[1, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[1, 0]
[0, 0]
[0, 0]
[0, 0]
[1, 0]
[0, 0]
[0, 0]
[0, 0]
interact
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[-1, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 1]


In [45]:
count

542